In [ ]:
import psycopg2
import pandas as pd
import requests
import csv
import yfinance as yf


print(psycopg2.__version__)
print(pd.__version__)
print(requests.__version__)
print(csv.__version__)
print(yf.__version__)


2.9.11 (dt dec pq3 ext lo64)
2.3.3
2.32.5
1.0
0.2.66


In [21]:
import requests
import csv
import yfinance as yf

# 修正 Yahoo Finance 符號 (BRK.B → BRK-B)
def fix_yahoo_symbol(symbol):
    return symbol.replace(".", "-")

# URL
url = "https://raw.githubusercontent.com/datasets/s-and-p-500-companies/refs/heads/main/data/constituents.csv"

# 下載資料
response = requests.get(url)
failed_symbols = []

if response.status_code == 200:
    content = response.content.decode('utf-8').splitlines()
    reader = csv.reader(content)
    next(reader)  # 跳過標題列

    # 關鍵：加入 encoding="utf-8"
    with open("sp500_symbols.txt", "w", encoding="utf-8") as file:
        for row in reader:
            symbol = row[0]
            name = row[1]
            yahoo_symbol = fix_yahoo_symbol(symbol)
            file.write(f"{yahoo_symbol} {name}\n")
            print(f"{yahoo_symbol} {name} have been saved.")

    print("All symbols saved to sp500_symbols.txt")

    # === 第二階段：用 yfinance 抓資料 ===
    with open("sp500_symbols.txt", "r", encoding="utf-8") as file:
        for line in file:
            line = line.strip()
            if not line:
                continue
            yahoo_symbol, name = line.split(" ", 1)  # 安全分割
            try:
                stock = yf.Ticker(yahoo_symbol)
                info = stock.info  # 觸發請求
                print(f"Success: {yahoo_symbol} ({name})")
            except Exception as e:
                print(f"Failed: {yahoo_symbol} ({name}) → {e}")
                failed_symbols.append(yahoo_symbol)
                
else:
    print("Failed to fetch CSV:", response.status_code)

# 寫入失敗清單
with open("failed_symbols.txt", "w", encoding="utf-8") as f:
    for s in failed_symbols:
        f.write(s + "\n")

print(f"Done! {len(failed_symbols)} symbols failed.")

MMM 3M have been saved.
AOS A. O. Smith have been saved.
ABT Abbott Laboratories have been saved.
ABBV AbbVie have been saved.
ACN Accenture have been saved.
ADBE Adobe Inc. have been saved.
AMD Advanced Micro Devices have been saved.
AES AES Corporation have been saved.
AFL Aflac have been saved.
A Agilent Technologies have been saved.
APD Air Products have been saved.
ABNB Airbnb have been saved.
AKAM Akamai Technologies have been saved.
ALB Albemarle Corporation have been saved.
ARE Alexandria Real Estate Equities have been saved.
ALGN Align Technology have been saved.
ALLE Allegion have been saved.
LNT Alliant Energy have been saved.
ALL Allstate have been saved.
GOOGL Alphabet Inc. (Class A) have been saved.
GOOG Alphabet Inc. (Class C) have been saved.
MO Altria have been saved.
AMZN Amazon have been saved.
AMCR Amcor have been saved.
AEE Ameren have been saved.
AEP American Electric Power have been saved.
AXP American Express have been saved.
AIG American International Group hav

2025-11-01 15:12:25,173 | ERROR | HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: WBA"}}}


Success: WBA (Walgreens Boots Alliance)
Success: WMT (Walmart)
Success: DIS (Walt Disney Company (The))
Success: WBD (Warner Bros. Discovery)
Success: WM (Waste Management)
Success: WAT (Waters Corporation)
Success: WEC (WEC Energy Group)
Success: WFC (Wells Fargo)
Success: WELL (Welltower)
Success: WST (West Pharmaceutical Services)
Success: WDC (Western Digital)
Success: WY (Weyerhaeuser)
Success: WSM (Williams-Sonoma, Inc.)
Success: WMB (Williams Companies)
Success: WTW (Willis Towers Watson)
Success: WDAY (Workday, Inc.)
Success: WYNN (Wynn Resorts)
Success: XEL (Xcel Energy)
Success: XYL (Xylem Inc.)
Success: YUM (Yum! Brands)
Success: ZBRA (Zebra Technologies)
Success: ZBH (Zimmer Biomet)
Success: ZTS (Zoetis)
Done! 0 symbols failed.


In [20]:

import psycopg2

# Database connection parameters
db_host = "localhost"
db_name = "bootcamp_2508_final_project"
db_user = "postgres"
db_password = "admin1234"

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host=db_host,
    database=db_name,
    user=db_user,
    password=db_password
)

# Create a cursor object
cursor = conn.cursor()

# Create table
create_table_query = """
CREATE TABLE IF NOT EXISTS sp500_symbols (
    symbol VARCHAR(10) primary key NOT NULL UNIQUE
);
"""
cursor.execute(create_table_query)
conn.commit()

# Open the file and read symbols
with open("sp500_symbols.txt", "r") as file:
    symbols = file.readlines()

# Insert symbols into the database
for symbol in symbols:
    symbol = symbol.strip()  # Remove any whitespace/newline characters
    try:
        cursor.execute("INSERT INTO sp500_symbols (symbol) VALUES (%s) ON CONFLICT (symbol) DO NOTHING;", (symbol,))
    except Exception as e:
        print(f"Error inserting {symbol}: {e}")

# Commit changes and close the connection
conn.commit()
cursor.close()
conn.close()

print("S&P 500 symbols have been stored in the PostgreSQL database.")

S&P 500 symbols have been stored in the PostgreSQL database.


In [ ]:
import psycopg2
import csv
import re
from datetime import datetime
import requests

# Database connection parameters
db_host = "localhost"
db_name = "bootcamp_2508_final_project"
db_user = "postgres"
db_password = "admin1234"

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host=db_host,
    database=db_name,
    user=db_user,
    password=db_password
)

# Create a cursor object
cursor = conn.cursor()

# Create table
create_table_query = """
create table IF NOT EXISTS sp500_info (
    symbol VARCHAR(10) primary key NOT NULL UNIQUE,
    security VARCHAR(255) NOT NULL,
    gics_sector VARCHAR(255) NOT NULL,
    gics_sub_industry VARCHAR(255) NOT NULL,
    headquarters_location VARCHAR(255) NOT NULL,
    date_added DATE NOT NULL,
    cik VARCHAR(20) NOT NULL,
    founded TEXT NOT NULL
);
"""
cursor.execute(create_table_query)
conn.commit()

# Load CSV data from URL
url = 'https://raw.githubusercontent.com/datasets/s-and-p-500-companies/refs/heads/main/data/constituents.csv'
# Fetch the CSV data
response = requests.get(url)


# Check if the request was successful
if response.status_code == 200:
    # Open a CSV file and read data
    # Decode the content and split into lines
    lines = response.content.decode('utf-8').splitlines()
    # Read the CSV data
    reader = csv.reader(lines)
    header = next(reader)  # Get the header row

    # Extract symbols and write to a .txt file
    with open("sp500_information.txt", "w", newline='', encoding='utf-8') as txt_file:
        # Optional: write the header as a comment line
        txt_file.write("# " + ",".join(header) + "\n")

        for row in reader:
            # Write the full row as a CSV line (comma-separated)
            txt_file.write(",".join(row) + "\n")

    print("All rows & columns have been saved to sp500_information.txt.")
else:
    print("Failed to fetch data:", response.status_code)

# Re-open the same data for DB insertion (re-use the same reader)
    # Reset the reader because we already consumed it above
lines = response.content.decode('utf-8').splitlines()
reader = csv.reader(lines)
next(reader)   # skip header again

# Insert data into the database
for row in reader:
    # Convert date from string to date object
    date_added = datetime.strptime(row[5], '%Y-%m-%d').date()
    
    # Clean the founded value
    founded_cleaned = re.sub(r'\s*\(.*\)', '', row[7])  # Remove anything in parentheses

    cursor.execute("""
        INSERT INTO sp500_info (symbol, security, gics_sector, gics_sub_industry, headquarters_location, date_added, cik, founded)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (symbol) DO NOTHING;
    """, (row[0], row[1], row[2], row[3], row[4], date_added, row[6], founded_cleaned))

# Commit changes and close the connection
conn.commit()
cursor.close()
conn.close()

print("S&P 500 stocks' information have been stored in the PostgreSQL database.")

All rows & columns have been saved to sp500_information.txt.
S&P 500 companies have been stored in the PostgreSQL database.


In [15]:
#!/usr/bin/env python3
"""
save_all_finnhub_to_one_json.py
Fast, safe, self-healing parallel fetch with auto-retry for empty data
"""
# --------------------------------------------------------------
# IMPORTS
# --------------------------------------------------------------
import json
import time
import logging
from pathlib import Path
from typing import Dict, Any, List
from concurrent.futures import ThreadPoolExecutor, as_completed   # ← enables parallel HTTP requests
from datetime import datetime
import requests
from requests.adapters import HTTPAdapter
from urllib3.util import Retry

# --------------------------------------------------------------
# CONFIGURATION
# --------------------------------------------------------------
FINNHUB_TOKEN = "d3rfce1r01qopgh8ajv0d3rfce1r01qopgh8ajvg"           # ← your Finnhub API key
FINNHUB_URL   = "https://finnhub.io/api/v1/stock/profile2"         # ← company profile endpoint
SYMBOLS_TXT   = "sp500_symbols.txt"                                # ← input: one ticker per line

# Output file with timestamp to avoid overwrites
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")                # ← e.g., 20251101_142153
OUTPUT_PATH = Path(f"C:/github/Bootcamp-Final-Project/python/sp500_finnhub_profiles_{timestamp}.json")

MAX_WORKERS = 12                                                   # ← safe concurrency (≤ pool size)
REQUESTS_PER_MIN = 55                                              # ← stay under 60/min free tier
DELAY_BETWEEN_BATCHES = 60 / REQUESTS_PER_MIN                      # ← ~1.09 sec avg per request

# --------------------------------------------------------------
# LOGGING
# --------------------------------------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[
        logging.FileHandler("fetch_one_json.log"),                 # ← log to file for audit
        logging.StreamHandler()                                    # ← also print to console
    ]
)
log = logging.getLogger(__name__)                                  # ← logger instance

# --------------------------------------------------------------
# HTTP SESSION WITH LARGE POOL + STRONG RETRY
# --------------------------------------------------------------
def get_session():
    session = requests.Session()                                   # ← reusable connection pool
    adapter = HTTPAdapter(
        pool_connections=20,                                       # ← allow 20 simultaneous hosts
        pool_maxsize=20,                                           # ← max 20 connections per host
        max_retries=Retry(
            total=5,                                               # ← retry up to 5 times
            backoff_factor=1.5,                                    # ← wait 1.5s, 3s, 6s, 12s, 24s
            status_forcelist=[429, 500, 502, 503, 504],            # ← retry on rate limit & server errors
            allowed_methods=["GET"]                                # ← only retry GET
        )
    )
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    session.headers.update({
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
    })                                                             # ← mimic real browser
    return session

# --------------------------------------------------------------
# HELPERS
# --------------------------------------------------------------
def load_symbols() -> List[str]:
    p = Path(SYMBOLS_TXT)
    if not p.is_file():
        log.error(f"{SYMBOLS_TXT} not found!")                     # ← abort if missing
        return []
    syms = [ln.strip().upper() for ln in p.open() if ln.strip() and not ln.startswith("#")]
    log.info(f"Loaded {len(syms)} symbols")                        # ← success message
    return syms

def finnhub_profile(symbol: str, session: requests.Session) -> Dict[str, Any]:
    sym = symbol.replace(".", "-")                                 # ← BRK.B → BRK-B (Finnhub format)
    url = f"{FINNHUB_URL}?symbol={sym}&token={FINNHUB_TOKEN}"
    try:
        r = session.get(url, timeout=10)                           # ← 10s timeout (was 7)
        if r.status_code == 200:
            data = r.json()
            return data if isinstance(data, dict) and data else {} # ← return only if valid
        if r.status_code == 429:
            wait = 15
            log.warning(f"Rate limit (429) for {symbol}, sleep {wait}s")
            time.sleep(wait)
            return finnhub_profile(symbol, session)                # ← recursive retry
        if r.status_code >= 500:
            log.warning(f"Server error {r.status_code} for {symbol}, will retry")
            return {}                                              # ← let main retry logic handle
        log.warning(f"HTTP {r.status_code} for {symbol}")
        return {}
    except requests.exceptions.RequestException as e:
        log.error(f"Request failed for {symbol}: {e}")
        return {}

# --------------------------------------------------------------
# MAIN
# --------------------------------------------------------------
def main() -> None:
    symbols = load_symbols()
    if not symbols:
        return

    OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)           # ← ensure output dir exists
    all_data: Dict[str, Any] = {}
    session = get_session()

    # ────── 1. FIRST PASS: Fast parallel fetch ──────
    log.info(f"1st PASS → {len(symbols)} symbols, {MAX_WORKERS} workers")
    start_time = time.time()
    fetched = 0

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        future_to_sym = {executor.submit(finnhub_profile, s, session): s for s in symbols}
        
        for future in as_completed(future_to_sym):
            sym = future_to_sym[future]
            fetched += 1
            profile = future.result()
            all_data[sym] = profile

            # Enforce average rate: 55 req/min
            expected = fetched * DELAY_BETWEEN_BATCHES
            elapsed = time.time() - start_time
            if elapsed < expected:
                time.sleep(expected - elapsed)

            status = "OK" if profile else "empty"
            log.info(f"[{fetched}/{len(symbols)}] {sym} → {status}")

    # ────── 2. AUTO RE-FETCH EMPTY PROFILES ──────
    empty_symbols = [s for s, d in all_data.items() if not d]
    log.info(f"First pass done. {len(empty_symbols)} symbols returned empty data.")

    if empty_symbols:
        log.info(f"2nd PASS → re-trying {len(empty_symbols)} empty symbols (patient mode)")
        for sym in empty_symbols:
            profile = finnhub_profile(sym, session)  # uses full retry logic
            if profile:
                all_data[sym] = profile
                log.info(f"  RECOVERED {sym}")
            else:
                log.info(f"  STILL EMPTY {sym}")
            time.sleep(3)  # gentle delay between retries

    # ────── 3. SAVE FINAL JSON ──────
    valid_count = sum(1 for d in all_data.values() if d)
    try:
        with OUTPUT_PATH.open("w", encoding="utf-8") as f:
            json.dump(all_data, f, indent=2, ensure_ascii=False)
        log.info(f"SUCCESS → {valid_count}/{len(all_data)} profiles saved to:")
        log.info(f"   {OUTPUT_PATH}")
    except Exception as e:
        log.error(f"Failed to write JSON: {e}")

    # Optional: save list of permanently empty symbols
    still_empty = [s for s, d in all_data.items() if not d]
    if still_empty:
        empty_file = OUTPUT_PATH.with_name(f"still_empty_{timestamp}.txt")
        empty_file.write_text("\n".join(still_empty), encoding="utf-8")
        log.info(f"{len(still_empty)} symbols still empty → {empty_file}")

if __name__ == "__main__":
    main()

2025-11-01 14:33:43,340 | INFO | Loaded 503 symbols
2025-11-01 14:33:43,341 | INFO | 1st PASS → 503 symbols, 12 workers
2025-11-01 14:33:44,534 | INFO | [1/503] AOS → OK
2025-11-01 14:33:45,524 | INFO | [2/503] ACN → OK
2025-11-01 14:33:46,615 | INFO | [3/503] AMD → OK
2025-11-01 14:33:47,705 | INFO | [4/503] AES → OK
2025-11-01 14:33:48,798 | INFO | [5/503] MMM → OK
2025-11-01 14:33:49,887 | INFO | [6/503] ABBV → OK
2025-11-01 14:33:50,978 | INFO | [7/503] ABT → OK
2025-11-01 14:33:52,069 | INFO | [8/503] ADBE → OK
2025-11-01 14:33:53,160 | INFO | [9/503] AKAM → OK
2025-11-01 14:33:54,251 | INFO | [10/503] ALB → OK
2025-11-01 14:33:55,342 | INFO | [11/503] LNT → OK
2025-11-01 14:33:56,433 | INFO | [12/503] ARE → OK
2025-11-01 14:33:57,524 | INFO | [13/503] AFL → OK
2025-11-01 14:33:58,615 | INFO | [14/503] ALGN → OK
2025-11-01 14:33:59,706 | INFO | [15/503] ALLE → OK
2025-11-01 14:34:00,796 | INFO | [16/503] ALL → OK
2025-11-01 14:34:01,889 | INFO | [17/503] GOOGL → OK
2025-11-01 14:3

In [9]:
#!/usr/bin/env python3
"""
load_json_to_postgres_clarity.py
-------------------------------
Load sp500_finnhub_profiles_*.json → PostgreSQL
Keep original numbers, just rename columns to show units
"""

import json
import logging
from pathlib import Path
from typing import Dict, Any
import psycopg2
import psycopg2.extras

# ----------------------------------------------------------------------
# CONFIGURATION
# ----------------------------------------------------------------------
JSON_GLOB = "C:/github/Bootcamp-Final-Project/python/sp500_finnhub_profiles_*.json"
DB_CONFIG = {
    "host": "localhost",
    "database": "bootcamp_2508_final_project",
    "user": "postgres",
    "password": "admin1234"
}

# ----------------------------------------------------------------------
# Logging
# ----------------------------------------------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[
        logging.FileHandler("load_json_clarity.log"),
        logging.StreamHandler()
    ]
)
log = logging.getLogger(__name__)

# ----------------------------------------------------------------------
# Create table with CLEAR column names
# ----------------------------------------------------------------------
def ensure_table(cur) -> None:
    cur.execute("""
    CREATE TABLE IF NOT EXISTS sp500_finnhub_profiles (
        symbol                     VARCHAR(10) PRIMARY KEY,
        country                    VARCHAR(100),
        currency                   VARCHAR(10),
        estimate_currency          VARCHAR(10),
        exchange                   VARCHAR(255),
        main_industry              VARCHAR(255),

        -- === UNITS CLEARLY STATED IN COLUMN NAME ===
        floating_share_millions    NUMERIC,   -- in millions (e.g., 14547.72 = 14.5B shares)
        market_cap_usd_millions    NUMERIC,   -- in USD millions (e.g., 4002453 = $4T)
        share_outstanding_millions NUMERIC,   -- in millions

        ipo_date                   DATE,
        logo                       TEXT,
        name                       VARCHAR(255),
        phone                      VARCHAR(50),
        ticker                     VARCHAR(10),
        weburl                     TEXT,
        raw_json                   JSONB      -- full original Finnhub response
    );
    """)
    log.info("Table sp500_finnhub_profile created with clear unit column names")

# ----------------------------------------------------------------------
# Upsert: map Finnhub fields → renamed columns (NO SCALING!)
# ----------------------------------------------------------------------
def upsert(cur, symbol: str, data: Dict[str, Any]) -> None:
    cur.execute("""
    INSERT INTO sp500_finnhub_profiles (
        symbol, country, currency, estimate_currency, exchange,
        main_industry,
        floating_share_millions, market_cap_usd_millions, share_outstanding_millions,
        ipo_date, logo, name, phone, ticker, weburl, raw_json
    ) VALUES (
        %(symbol)s, %(country)s, %(currency)s, %(estimateCurrency)s,
        %(exchange)s, %(finnhubIndustry)s,
        %(floatingShare)s, %(marketCapitalization)s, %(shareOutstanding)s,
        %(ipo)s::date, %(logo)s, %(name)s, %(phone)s,
        %(ticker)s, %(weburl)s, %(raw_json)s
    )
    ON CONFLICT (symbol) DO UPDATE SET
        country                    = EXCLUDED.country,
        currency                   = EXCLUDED.currency,
        estimate_currency          = EXCLUDED.estimate_currency,
        exchange                   = EXCLUDED.exchange,
        main_industry              = EXCLUDED.main_industry,
        floating_share_millions    = EXCLUDED.floating_share_millions,
        market_cap_usd_millions    = EXCLUDED.market_cap_usd_millions,
        share_outstanding_millions = EXCLUDED.share_outstanding_millions,
        ipo_date                   = EXCLUDED.ipo_date,
        logo                       = EXCLUDED.logo,
        name                       = EXCLUDED.name,
        phone                      = EXCLUDED.phone,
        ticker                     = EXCLUDED.ticker,
        weburl                     = EXCLUDED.weburl,
        raw_json                   = EXCLUDED.raw_json;
    """, {
        "symbol": symbol,
        "country": data.get("country"),
        "currency": data.get("currency"),
        "estimateCurrency": data.get("estimateCurrency"),
        "exchange": data.get("exchange"),
        "finnhubIndustry": data.get("finnhubIndustry"),

        # Direct mapping — NO division!
        "floatingShare": data.get("floatingShare"),
        "marketCapitalization": data.get("marketCapitalization"),
        "shareOutstanding": data.get("shareOutstanding"),

        "ipo": data.get("ipo"),
        "logo": data.get("logo"),
        "name": data.get("name"),
        "phone": data.get("phone"),
        "ticker": data.get("ticker"),
        "weburl": data.get("weburl"),
        "raw_json": psycopg2.extras.Json(data)
    })

# ----------------------------------------------------------------------
# Main
# ----------------------------------------------------------------------
def main() -> None:
    json_files = list(Path("C:/github/Bootcamp-Final-Project/python").glob("sp500_finnhub_profiles_*.json"))
    if not json_files:
        log.error("No JSON file found!")
        return
    json_path = max(json_files, key=lambda p: p.stat().st_mtime)
    log.info(f"Loading from: {json_path}")

    try:
        with json_path.open("r", encoding="utf-8") as f:
            all_data = json.load(f)
        log.info(f"Loaded {len(all_data)} profiles")
    except Exception as e:
        log.error(f"JSON read error: {e}")
        return

    conn = None
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cur = conn.cursor()
        ensure_table(cur)

        count = 0
        for symbol, profile in all_data.items():
            if not profile:
                log.warning(f"Skipping empty: {symbol}")
                continue
            upsert(cur, symbol, profile)
            count += 1
            if count % 50 == 0:
                conn.commit()
                log.info(f"Committed {count} rows")

        conn.commit()
        log.info(f"SUCCESS: {count} records loaded with clear column names")
        cur.close()
    except Exception as e:
        log.error(f"DB error: {e}")
        if conn:
            conn.rollback()
    finally:
        if conn:
            conn.close()

if __name__ == "__main__":
    main()

2025-11-01 13:48:57,498 | INFO | Loading from: C:\github\Bootcamp-Final-Project\python\sp500_finnhub_profiles_20251101_123857.json
2025-11-01 13:48:57,500 | INFO | Loaded 503 profiles
2025-11-01 13:48:57,649 | INFO | Table sp500_finnhub_profile created with clear unit column names
2025-11-01 13:48:57,659 | INFO | Committed 50 rows
2025-11-01 13:48:57,660 | WARNING | Skipping empty: AVB
2025-11-01 13:48:57,669 | INFO | Committed 100 rows
2025-11-01 13:48:57,679 | INFO | Committed 150 rows
2025-11-01 13:48:57,689 | INFO | Committed 200 rows
2025-11-01 13:48:57,698 | INFO | Committed 250 rows
2025-11-01 13:48:57,709 | INFO | Committed 300 rows
2025-11-01 13:48:57,716 | INFO | Committed 350 rows
2025-11-01 13:48:57,727 | INFO | Committed 400 rows
2025-11-01 13:48:57,734 | INFO | Committed 450 rows
2025-11-01 13:48:57,744 | INFO | Committed 500 rows
2025-11-01 13:48:57,745 | INFO | SUCCESS: 502 records loaded with clear column names
